<h1> Eksploracja masywnych danych </h1>
<h2> Projekt Python - analiza sentymentu </h2>

<h3> 1. Wstępna analiza danych i preprocessing </h3>

In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
df = pd.read_csv('reviews_train.csv')
print(df.shape)
df = df.dropna()
print(df.shape)
df.head(15)

(555791, 9)
(555745, 9)


,reviewerID,asin,reviewerName,helpful,reviewText,summary,unixReviewTime,reviewTime,score
0,A35C43YE9HU9CN,B0064X7B4A,Joan Miller,"[0, 0]",I have decided not to play this game. I can't...,Friends,1396396800,"04 2, 2014",1.0
1,AHFS8CGWWXB5B,B00H1P4V3E,WASH ST. GAMER,"[3, 4]",The Amazon Appstore free app of the day for Ju...,"Amazon Makes This ""Longest Spring Ever"" for Fi...",1402272000,"06 9, 2014",2.0
2,A3EW8OTQ90NVHM,B00CLVW82O,Kindle Customer,"[0, 4]",this game was so mush fun I wish I could play ...,best,1368921600,"05 19, 2013",5.0
3,AJ3GHFJY1IUTD,B007T9WVKM,BrawlMaster4,"[0, 2]","Its pretty fun and very good looking, but you...",Fun Game,1350172800,"10 14, 2012",5.0
4,A3JJGBS4EL603S,B00J206J5E,"K. Wilson ""thesupe""","[0, 0]",good graphics; immersive storyline; hard to st...,great game!,1396915200,"04 8, 2014",5.0
5,A3RL7Y2FJBDHJ0,B006H7TC3Q,hi,"[2, 5]",its very good.u use fotos on ur device and it ...,very good,1337817600,"05 24, 2012",5.0
6,AUHVMC0PURGO8,B006R6VG9K,A.Mccullough,"[0, 0]",the game is very fun and fast paced. It also k...,fun and fast paced,1401926400,"06 5, 2014",5.0
7,A1Z37DUIWXJNLN,B00B63HT8Q,"Julie Quick ""Beach Bum Wannabe""","[0, 0]",great app! A quick look at the weather... not...,A great alternative to the current (stupid) ve...,1377388800,"08 25, 2013",4.0
8,AF7ZE5MRM6CW2,B00BL0I7WG,T.dd,"[0, 0]",So fare I like it haven't had it long enough t...,easy fun,1369612800,"05 27, 2013",5.0
9,A1TTH51E2651BJ,B00GRXA7GG,Joni,"[0, 0]",This classic Mahjong comes with nice graphics ...,Mahjong Premium,1394841600,"03 15, 2014",5.0


In [3]:
score_counts = df.value_counts('score')
score_frequencies = score_counts/np.sum(score_counts)
score_frequencies = score_frequencies.sort_index()
score_frequencies

score
1.0    0.108215
2.0    0.060702
3.0    0.114966
4.0    0.209690
5.0    0.506427
dtype: float64

<h4> Interpretacja danych </h4>

Wstępne przejrzenie przykładowych danych dostępnych w dokumencie pozwala na wstępną ocenę ich użyteczności, a także potencjalnych trudności w ich interpretacji.

Już pierwszy rzut oka na dane wskazuje, jak wielkie znaczenie może mieć (przeważnie krótkie) podsumowanie obecne w kolumnie "summary". Niektórzy użytkownicy wprost umeszczają w niej swoje oceny, łatwe do zinterpretowania na kilkustopniowej skali.

Najbardziej wiarygodną podstawą do klasyfikacji, pod warunkiem dobrej jakości modelu, powinien być pełen tekst recenzji. W przypadku problemu analizy sentymentu, kwestia wykorzystania "stop words" jest trudna. Wiele słów (np. zaprzeczeń) są kluczowe dla prawidłowej klasyfikacji wypowiedzi i nie należy ich usuwać. Być może jednak warto pozbyć się innych, nie niosących takich informacji.

Wiele słów wprost odwraca znaczenie dalszej części zdania. Nie sa to tylko trywialne przypadki, jak te z zaprzeczeniem (np. "not"). Już w niewielkim wycinku danych widoczny jest taki przykład - "A great ALTERNATIVE to (...) STUPID...". Pokazuje to, że słowa, które w ten sposób działają na interpretację zdania są powszechne i nietrywialne - odpowiednio dobrany system uczący powinien zwracać uwagę na kolejność słów lub choćby rozróżniać kolejne zdania. By sprawdzić, jak ważna jest ta zdolność, najpierw zostanie przeprowadzona klasyfikacja z użyciem modelu, który jej nie posiada - naiwnego klasyfikatora bayesowskiego.

Wykorzystanie głosów innych użytkowników na temat użyteczności opinii może mijać się z celem. Na podstawie niewielkej próbki zdaje się, że liczba głosów pod opiniami wynosi zero lub jest niewielka. Można się spodziewać, że pozytywne głosy zbiorą te recenzję, których ocena zgodna jest z oceną większości użytkowników, a opinie niepopularne zbiorą głosy negatywne. Obecne w zbiorze opinie dotyczą jednak różnych produktów, wobec tego prawdopodobnie nie ma sensu śledzić tego ogólnego trendu, by wydobyć dodatkowe informacje.

Problemem, który należy mieć na uwadze, jest niezbilansowanie zbioru pod względem ocen. Ponad połowa ocen jest równa 5, a ponad 70% jest równa 4 lub 5. Można więc przyjąć, że klasyfikatory, które wnoszą pewną jakość do klasyfikacji (oczywiście z perspektywy trafności, a nie innych metryk) to takie, które klasyfikują prawidłowo więcej niż 50% przypadków.

Pierwsze podejście do problemu zostanie wykonane w dwóch wariantach - z predykcją jednej z 5 ocen lub predykcją, czy ocena jest pozytywna (4-5) lub negatywna (1-3).

In [4]:
binary_grade_df = df.copy()
binary_grade_df['score'] = np.where(binary_grade_df['score'] > 3, 1, 0)
binary_grade_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,summary,unixReviewTime,reviewTime,score
0,A35C43YE9HU9CN,B0064X7B4A,Joan Miller,"[0, 0]",I have decided not to play this game. I can't...,Friends,1396396800,"04 2, 2014",0
1,AHFS8CGWWXB5B,B00H1P4V3E,WASH ST. GAMER,"[3, 4]",The Amazon Appstore free app of the day for Ju...,"Amazon Makes This ""Longest Spring Ever"" for Fi...",1402272000,"06 9, 2014",0
2,A3EW8OTQ90NVHM,B00CLVW82O,Kindle Customer,"[0, 4]",this game was so mush fun I wish I could play ...,best,1368921600,"05 19, 2013",1
3,AJ3GHFJY1IUTD,B007T9WVKM,BrawlMaster4,"[0, 2]","Its pretty fun and very good looking, but you...",Fun Game,1350172800,"10 14, 2012",1
4,A3JJGBS4EL603S,B00J206J5E,"K. Wilson ""thesupe""","[0, 0]",good graphics; immersive storyline; hard to st...,great game!,1396915200,"04 8, 2014",1


In [5]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Grzegorz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Grzegorz\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
example_sentence = binary_grade_df.iloc[0,4]
tokens = word_tokenize(binary_grade_df.iloc[0,4])
tokens_pretty = ' '.join(tokens)
lemmas = ' '.join([lemmatizer.lemmatize(t) for t in tokens])
stems = ' '.join([stemmer.stem(t) for t in tokens])

print(example_sentence, tokens_pretty, lemmas, stems, sep='\n')

I have decided not to play this game.  I can't keep track of everyone, or play often enough to make it fun.
I have decided not to play this game . I ca n't keep track of everyone , or play often enough to make it fun .
I have decided not to play this game . I ca n't keep track of everyone , or play often enough to make it fun .
i have decid not to play thi game . i ca n't keep track of everyon , or play often enough to make it fun .


Jak widać na powyższym przykładzie, stemming upraszcza tekst nieco bardziej niż lematyzacja. Z jednej strony, wpływa to na lepsze uogólnienie słów, z drugiej - może zniekształcić kontekst zdania.

<h3> 2. Wstępna klasyfikacja </h3>


In [7]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

Ze względu na rodzaj problemu (kolejnym etykietom odpowiadają wartości uporządkowane liniowo), oprócz metryk typowych dla klasyfikacji wieloklasowej, wykorzystane zostaną metryki typowe dla regresji. W poniższej kolumnie zdefiniowano uogólnienie metryk f1 dla klasyfikacji wieloklasowej oraz średni błąd absolutny i błąd średniokwadratowy, wyliczane z macierzy pomyłek.

In [8]:
recall = lambda confusion_mx, clazz: confusion_mx[clazz, clazz]/np.sum(confusion_mx[:, clazz])
precision = lambda confusion_mx, clazz: confusion_mx[clazz, clazz]/np.sum(confusion_mx[clazz, :])
micro_f1_score = lambda confusion_mx, clazz: 2/(precision(confusion_mx, clazz)**-1 * recall(confusion_mx, clazz)**-1)
macro_f1_score = lambda confusion_mx: np.mean([micro_f1_score(confusion_mx, clazz) for clazz in range(confusion_mx.shape[0])])

accuracy = lambda confusion_mx: np.sum(np.diag(confusion_mx))/np.sum(confusion_mx)

error_matrix = np.abs(np.arange(5)[:, np.newaxis] - np.arange(5))
mae = lambda confusion_mx: np.sum(confusion_mx * error_matrix / np.sum(confusion_mx))
mse = lambda confusion_mx: np.sum(confusion_mx * error_matrix**2 /  np.sum(confusion_mx))

Gdyby klasyfikator losowo przypisywał etykiety, ale z prawdopodobieństwem proporcjonalnym do liczebności klasy decyzyjnej, wówczas f1 score wyniósłby:

In [9]:
from scipy.optimize import minimize
dummy_confusion_mx = lambda freqs, probs: freqs[:, np.newaxis] * probs

freqs = np.array(score_frequencies)
probs = freqs

macro_f1_score(dummy_confusion_mx(freqs, probs))

0.13162005051281517

Dla wygody modelowania wyników klasyfikatora niewnoszącego wartości informacyjnej, a także późniejszego przetwarzania licznych wyników z walidacji jedynie na podstawie macierzy pomyłek, zdefiniowano funkcje accuracy i f1_score wyliczane z macierzy pomyłek.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
reviewText = np.array(df['reviewText'])
summaryText = np.array(df['summary'])
labels_multi = np.array(df['score'])
labels_binary = np.array(binary_grade_df['score'])

<h4> Wstępna optymalizacja modelu </h4>

Zaproponowano dwa sposoby wektoryzacji, z wykorzystaniem klasy CountVectorizer (zliczanie słów) oraz TfidfVectorizer (przypisującym wyższą wagę za wystąpienie termów wówczas, gdy znajdą się one w mniejszej liczbie przykładów ze zbioru - realizuje to podobną ideę, co wycinanie stop words, tylko w bardziej "miękki" sposób). <br>
Zarówno dla jakości, jak i prędkości predykcji, konieczne okazało się ograniczenie liczby atrybutów: narzucenie limitu maksymalnej liczby najczęściej występujących słów lub minimalną liczbę wystąpień słowa w zborze danych. W drugim podejściu, optymalizacja okazała się łatwiejsza. <br>
Zgodnie ze wskazówkami dotyczącymi problemu analizy sentymentu, lepsze wyniki osiągnięto w podejściu bez usuwania stop words. <br>
Ze względu na znaczny koszt tokenizacji masywnego zbioru (nie jest to operacja, którą można zwektoryzować, a wyjściowe dane mają typ "object"), pozostano przy natywnej tokenizacji wykorzystywanej przez pakiet sklearn.

Zoptymalizowany model osiąga lepsze wyniki niż "ślepy" klasyfikator, preferujący zawsze klasę większościową. Niezoptymalizowane modele daleko odbiegały od tych wyników - w obecności zbędnych, mało informatywnych słów, klasyfikacja była znacznie zaburzona i klasa większościowa nie zawsze nawet tą najczęściej wybieraną przez model.

In [11]:
vectorizer1 = CountVectorizer(min_df=250)
vectorizer2 = TfidfVectorizer(min_df=250)
#vectorizer1 = CountVectorizer(min_df=250, stop_words='english')
#vectorizer2 = TfidfVectorizer(min_df=250, stop_words='english')
datapoints = 50000
xdata1 = vectorizer1.fit_transform(reviewText[:datapoints])
xdata2 = vectorizer2.fit_transform(reviewText[:datapoints])

<h4> Dobór vectorizera </h4>

Ze względu na znaczną ilość danych, wstępny wybór modelu został wykonany przez iteracyjny dobór parametru min_df i porównanie działania obu klas Vectorizer. Ponieważ próbka 50000 punktów danych to nieznaczna część całego zbioru (mniej niż 10%), nie wydzielono zbioru testowego. Zbiór walidacyjny i testowy zostaną rozróżnione przy finalnej parametryzacji modelu dla całego datasetu.

In [12]:
X_train1, X_val1, X_train2, X_val2, y_train, y_val, binary_y_train, binary_y_val = train_test_split(
    xdata1, xdata2, df['score'][:datapoints], binary_grade_df['score'][:datapoints], test_size=0.2, random_state=1)

In [13]:
binary_classifier1 = GaussianNB()
binary_classifier2 = GaussianNB()
multiclass_classifier1 = GaussianNB()
multiclass_classifier2 = GaussianNB()

binary_classifier1.fit(X_train1.toarray(), binary_y_train)
binary_classifier2.fit(X_train2.toarray(), binary_y_train)
multiclass_classifier1.fit(X_train1.toarray(), y_train)
multiclass_classifier2.fit(X_train2.toarray(), y_train)

y_pred_bin1 = binary_classifier1.predict(X_val1.toarray())
y_pred_bin2 = binary_classifier2.predict(X_val2.toarray())
y_pred_multi1 = multiclass_classifier1.predict(X_val1.toarray())
y_pred_multi2 = multiclass_classifier2.predict(X_val2.toarray())
 
res_bin_1 = confusion_matrix(binary_y_val, y_pred_bin1)
res_bin_2 = confusion_matrix(binary_y_val, y_pred_bin2)
res_multi_1 = confusion_matrix(y_val, y_pred_multi1)
res_multi_2 = confusion_matrix(y_val, y_pred_multi2)

In [14]:
print('Count vectorizer accuracies: ', accuracy(res_bin_1), accuracy(res_multi_1))
print('Count vectorizer accuracies: ', accuracy(res_bin_2), accuracy(res_multi_2))

Count vectorizer accuracies:  0.7656 0.4815
Count vectorizer accuracies:  0.7485 0.4456


CountVectorizer odnosi nieco lepsze rezultaty na zbiorze walidacyjnym. Zostanie wykorzystany w dalszej części eksperymentów.

<h4> Uogólnienie testów </h4>

Ze względu na fakt, że klasyfikator nie jest w stanie przetwarzać wektorów rzadkich, dane uczące mogą mieć rozmiar wielokrotne przekraczający dostępną pamięć operacyjną. Zaletą staje się tutaj prostota klasyfikatora - uczy się on przez zliczanie, jednorazowo przechodząc po danych. Wystarczy zatem wykonać jedną iterację, przetwarzając kolejne fragmenty zbioru na macierz pełną.

Ponadto, schemat każdego eksperymentu z wektoryzacją i klasyfikacją tekstów będzie wyglądał podobnie, jak w przykładzie powyżej. Czas zdefiniować całą procedurę. Potrzebna będzie możliwość doboru różnych wektoryzacji, różnych klasyfikatorów, różnych danych i etykiet (binarnych lub wieloklasowych), a także wybór między wektorem danych w postaci pełnej i rzadkiej. Drobna nadmiarowość (na przykład każdorazowe wyznaczanie trzech zbiorów - uczącego, walidacyjnego i testowego) zostanie zrekompensowana przez wygodę użycia. 

In [15]:
def experiment(vectorizer, classifier, data, labels, number_of_datapoints=None, batch_size=None, use_test_set=False, transform_sparse_data=False, return_model=False):
    
    if vectorizer is not None:
        # Wektoryzacja - eksperyment na niepełnym zbiorze danych
        if number_of_datapoints is None:
            xdata = vectorizer.fit_transform(data)
        # ...na pełnym zbiorze
        else:
            xdata = vectorizer.fit_transform(data[:number_of_datapoints])
            labels = labels[:number_of_datapoints]
    else:
        if number_of_datapoints is None:
            xdata = data
        else:
            xdata = vectorizer[:number_of_datapoints]
            labels = labels[:number_of_datapoints]
        
    # deterministyczny podział na zbiór uczący, walidacyjny i testowy
    x_train, x_test, y_train, y_test = train_test_split(xdata, labels, test_size=0.2, random_state=111)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=7)
    
    # uczenie i walidacja/testowanie
    if batch_size is None:
        # wymagane przez klasyfikator NB, niewymagane przez regresję logistyczną
        if transform_sparse_data:
            x_train = x_train.toarray()
            x_val = x_val.toarray()
            x_test = x_test.toarray()
        classifier.fit(x_train, y_train)
        # ostateczna próba będzie wykonywana na zbiorze testowym
        if use_test_set:
            result = confusion_matrix(y_val, classifier.predict(x_val))
        # większość (domyślnie) na walidacyjnym
        else:
            result = confusion_matrix(y_val, classifier.predict(x_val))
    # wersja z batch_size potrzebuje etykiet klas do i wykorzystuje metodę partial_fit
    else:
        classes = np.unique(y_train)
        for i in range(0, y_train.shape[0], batch_size):
            if transform_sparse_data:
                x_train_batch = x_train[i:i+batch_size].toarray()
            else:
                x_train_batch = x_train[i:i+batch_size]
            classifier.partial_fit(x_train_batch, y_train[i:i+batch_size], classes)
        if use_test_set:
            x_check = x_test
            y_check = y_test
        else:
            x_check = x_val
            y_check = y_val
        result = None
        for i in range(0, y_check.shape[0], batch_size):
            if transform_sparse_data:
                x_check_batch = x_check[i:i+batch_size].toarray()
            else:
                x_check_batch = x_check[i:i+batch_size]
            new_result = confusion_matrix(y_check[i:i+batch_size], classifier.predict(x_check_batch))
            if result is None:
                result = new_result
            else:
                result += new_result
    if return_model:
        return result, classifier
    return result

In [16]:
experiment(CountVectorizer(min_df=250), GaussianNB(), summaryText, labels_multi, number_of_datapoints=10000,
           transform_sparse_data=True)

array([[153,   7,   7,   2,  10],
       [ 72,   7,   2,   1,   4],
       [108,  11,  20,  15,  26],
       [143,   6,  20,  48, 111],
       [368,  20,  12,  49, 378]], dtype=int64)

In [17]:
# wybór modelu na zbiorze walidacyjnym wybierany pod kątem głównego zadania - klasyfikacji pięcioklasowej

results = []

batch_size = 10000
for min_df in range(250, 3000, 250):
    res = experiment(CountVectorizer(min_df=min_df), GaussianNB(), reviewText, labels_multi,
                     transform_sparse_data=True, batch_size=batch_size)
    print(min_df, accuracy(res))
    results.append((min_df, res))

250 0.45734367971210077
500 0.48575123706702655
750 0.49794197031039134
1000 0.5058816914080072
1250 0.5070625281151597
1500 0.5113697705802969
1750 0.5112910481331534
2000 0.5108187134502924
2250 0.5068713450292398
2500 0.506264057579847
2750 0.50944669365722


Próba na zbiorze testowym:

In [18]:
test_res = experiment(CountVectorizer(min_df=1500), GaussianNB(), reviewText, labels_multi,
                      transform_sparse_data=True, batch_size=batch_size, use_test_set=True)

In [19]:
print(test_res)
print('accuracy: ', accuracy(test_res))
print('f1_score: ', macro_f1_score(test_res))
print('mae :', mae(test_res))
print('mse :', mse(test_res))

[[ 7990  1406   586   344  1594]
 [ 2939  1233   743   507  1271]
 [ 3510  1546  1950  1785  4014]
 [ 3161  1383  2041  4522 12475]
 [ 5537  1885  2187  5921 40619]]
accuracy:  0.5066532312481444
f1_score:  0.34310851521789626
mae : 0.9153208755814267
mse : 2.2640689524872015


In [20]:
batch_size = 10000
for min_df in range(150, 1500, 150):
    res = experiment(CountVectorizer(min_df=min_df), GaussianNB(), summaryText, labels_multi,
                     transform_sparse_data=True, batch_size=batch_size)
    print(min_df, accuracy(res))

150 0.4565901934322987
300 0.4915991902834008
450 0.5299257759784075
600 0.5232905982905983
750 0.4799257759784076
900 0.4730656770130454
1050 0.46920827710301394
1200 0.4626293297345929
1350 0.45516194331983806


In [21]:
summary_test_res = experiment(CountVectorizer(min_df=500), GaussianNB(), summaryText, labels_multi,
                              transform_sparse_data=True, batch_size=batch_size, use_test_set=True)

Pomimo znacznie mniejszej liczby słów, podsumwania również są bardzo dobrą podstawą do predykcji (co jest oczekiwane od podsumowania). Na zbiorze testowym:

In [22]:
print(summary_test_res)
print('accuracy: ', accuracy(summary_test_res))
print('f1_score: ', macro_f1_score(summary_test_res))
print('mae :', mae(summary_test_res))
print('mse :', mse(summary_test_res))

[[ 6852  1305   285   301  3177]
 [ 2541  1569   494   353  1736]
 [ 2355  2772  2040  1404  4234]
 [ 2850  1707  1359  3859 13807]
 [ 4871  2287  1190  4231 43570]]
accuracy:  0.5208323961529119
f1_score:  0.34858359446700715
mae : 0.9165624522037985
mse : 2.3548839845612646


<h4> Random forest </h4>

Klasyfikator RandomForest powinien poradzić sobie lepiej niż Naiwny Bayes. Proces uczenia okazał się jednak zbyt kosztowny, by wykorzystać pełny zbiór danych. Klasyfikator uczony na niewielkim zbiorze i tak daje lepsze wyniki niż GaussianNB.

Powodem wysokiego kosztu obliczeniowego jest prawdopodobnie rzadki charakter danych, przez który głębokie, silnie dopasowane do zbioru uczącego drzewa całkiem dobrze wypadają na zbiorze testowym (>60%). Ograniczenie głębokości drzew skutkuje szybszym uczeniem, ale także znacznym spadkiem jakości predykcji.

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
res = experiment(CountVectorizer(min_df=200), RandomForestClassifier(n_estimators=200, n_jobs=-1), reviewText,
                     labels_multi, number_of_datapoints=20000)
print(accuracy(res))

0.5746875


In [25]:
res = experiment(CountVectorizer(min_df=500), RandomForestClassifier(n_estimators=500,  max_depth=15,
                n_jobs=-1, min_samples_split=10), reviewText, labels_multi, number_of_datapoints=20000)
print(accuracy(res))

0.5346875


<h4> Regresja logistyczna </h4>

Skoro klasyfkator złożony jest trudny do wykorzystania, konkurencyjnym modelem może okazać się regresja logistyczna, która powinna nauczyć się znacznie szybciej, a zwrócić lepsze wyniki niż GaussianNB. Podobnie jak RandomForest, może korzystać z macierzy rzadkich, dlatego nie trzeba preztwarzać ich na pełne macierze.

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
# tekst recenzji

res = experiment(CountVectorizer(min_df=1000), LogisticRegression(max_iter=100), reviewText, labels_multi)
print(accuracy(res))

0.627519118308592


C:\Users\Grzegorz\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [28]:
# podsumowanie recenzji

res = experiment(CountVectorizer(min_df=100), LogisticRegression(max_iter=100), summaryText, labels_multi)
print(accuracy(res))

0.6073549257759784


C:\Users\Grzegorz\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [29]:
'''
x_train, x_test, x_train_summary, x_test_summary, y_train, y_test = train_test_split(reviewText, summaryText, labels_multi, test_size=0.2, random_state=111)
x_train, x_val, x_train_summary, x_val_summary, y_train, y_val = train_test_split(x_train, x_train_summary, y_train, test_size=0.2, random_state=7)
'''

Zwięzłe podsumowana recenzji, choć są bardzo krótkie, pozwalają na niemal równie dobrą predykcję. Możliwe, że ich połączenie da lepsze wyniki - trudne do sklasyfikowania podsumowania mogą towarzyszyć łatwym tekstom do recenzji lub na odwrót.

Warto wykorzystać także informację o wzajemnym położeniu słów - n-gramy lub skip-gramy. Ze względu na duży zbiór danych, przetwarzanie tekstów i przechowywanie ich w postaci macierzy gęstych może być kosztowne (w tej postacicały zbiór nie mieści się jednocześnie w pamięci). Przetwarzanie tekstu powinno więc skutkować otrzymaniem macierzy rzadkiej, w miarę możliwości z odrzuceniem rzadko występujących (nieinformatywnych) zbitków słów, dlatego wykorzystana zostanie modyfkacja klasy CountVectrizer.

Wykorzystana implementacja może zostać łatwo uogólniona na różne definicje skip-gramów lub n-gramów. Ze zbioru uczącego zostaną wyciągnięte 2-gramy.

In [31]:
try:
    from toolz import itertoolz, compose
except:
    !pip install toolz
    from toolz import itertoolz, compose
from toolz.curried import map as cmap, sliding_window, pluck

In [32]:
# źródło: StackOverflow

class SkipGramVectorizer(CountVectorizer):
    def build_analyzer(self):    
        preprocess = self.build_preprocessor()
        stop_words = self.get_stop_words()
        tokenize = self.build_tokenizer()
        return lambda doc: self._word_skip_grams(
                compose(tokenize, preprocess, self.decode)(doc),
                stop_words)

    def _word_skip_grams(self, tokens, stop_words=None):
        # handle stop words
        if stop_words is not None:
            tokens = [w for w in tokens if w not in stop_words]

        return compose(cmap(' '.join), pluck([0, 1]), sliding_window(2))(tokens)

In [33]:
text = ['Kowalski ukradł krzesło i poszedł siedzieć']

vect = SkipGramVectorizer()
vect.fit(text)
vect.get_feature_names()

['kowalski ukradł', 'krzesło poszedł', 'poszedł siedzieć', 'ukradł krzesło']

In [34]:
ngramText = SkipGramVectorizer(min_df=100)
skipgrams = ngramText.fit_transform(reviewText)

In [35]:
# vectorizer=None - wektoryzacja została już wykonana
res = experiment(None, LogisticRegression(max_iter=100), skipgrams, labels_multi)

C:\Users\Grzegorz\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [36]:
print(accuracy(res))

0.6345029239766082


In [37]:
ngramSummary = SkipGramVectorizer(min_df=100)
skipgramsSummary = ngramSummary.fit_transform(summaryText)

In [38]:
res = experiment(None, LogisticRegression(max_iter=100), skipgramsSummary, labels_multi)
print(accuracy(res))

0.5707939721097616


C:\Users\Grzegorz\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


<h3> Końcowy model </h3>

Do klasyfikacji zostanie wykorzystana regresja logistyczna z parametrami pozyskanymi z kolumn reviewText i summary.

In [43]:
import scipy
from scipy.sparse import hstack
textVectorizer = CountVectorizer(min_df=1000)
summaryVectorizer = CountVectorizer(min_df=100)
textData = textVectorizer.fit_transform(reviewText)
summaryData = summaryVectorizer.fit_transform(summaryText)

In [44]:
fullData = hstack([textData, summaryData, skipgrams, skipgramsSummary])
res, model = experiment(None, LogisticRegression(max_iter=100), fullData, labels_multi, return_model=True)
print(accuracy(res))

0.6795096716149348


C:\Users\Grzegorz\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [47]:
print(res)
print('accuracy: ', accuracy(res))
print('f1_score: ', macro_f1_score(res))
print('mae :', mae(res))
print('mse :', mse(res))

[[ 7406   755   693   206   667]
 [ 1987  1168  1522   317   457]
 [ 1030   703  4617  2205  1570]
 [  325   145  1763  7177  9139]
 [  509    96   607  3802 40054]]
accuracy:  0.6795096716149348
f1_score:  0.6822640928391778
mae : 0.43360323886639673
mse : 0.7635627530364373


In [45]:
import pickle

with open('clfasifier.pickle', 'wb') as f:
    pickle.dump(model, f)

with open('vectorizers.pickle', 'wb') as f:
    pickle.dump([textVectorizer, summaryVectorizer, ngramText, ngramSummary], f)